## Capstone Project Notebook

### Uploading Data form Wiki Page

In [1]:
# importing libraries 
import pandas as pd
import numpy as np
!pip install lxml
!pip install html5lib
!pip install BeautifulSoup4

!pip install geopy
!pip install geocoder
from geopy.geocoders import Nominatim
import geocoder
import requests # Handles requests

import json # to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library


print('Libraries Imported')
print("Hello Capstone Project Course!")

     |████████████████████████████████| 102kB 5.4MB/s ta 0:00:011
     |████████████████████████████████| 102kB 9.6MB/s eta 0:00:01
Libraries Imported
Hello Capstone Project Course!


#### Scraping Webpage

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
# read files into a pandas dataframe

from pandas.io.html import read_html 

# Get page and the table from page using class 'wikitable'

page = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Seattle'
wikitables = read_html(page, attrs={"class": "wikitable"})

print('Data downloaded!')

Data downloaded!


In [4]:
wikitables[0].head()

,Unnamed: 0,Neighborhood name,Within larger district,Annexed[41],Locator map,Street map,Image,Notes
0,1,North Seattle,Seattle,Various,NaN,NaN,NaN,North of the Lake Washington Ship Canal[42]
1,2,Broadview,North Seattle[42],1954[43],NaN,NaN,NaN,[44]
2,3,Bitter Lake,North Seattle[42],1954[43],NaN,NaN,NaN,[45]
3,4,North Beach / Blue Ridge,North Seattle[42],"1940,[43] 1954[43]",NaN,NaN,NaN,[46]
4,5,Crown Hill,North Seattle[42],"1907,[47] 1952,[43] 1954[43]",NaN,NaN,NaN,[48]


In [5]:
# Define dataframe as df and clean df
df=wikitables[0]
df=df.drop(['Unnamed: 0','Annexed[41]',"Locator map","Street map","Image","Notes"], axis=1)

print(df.shape)
df.head()

(127, 2)


,Neighborhood name,Within larger district
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,North Beach / Blue Ridge,North Seattle[42]
4,Crown Hill,North Seattle[42]


In [6]:
# Rename second columns to District!
df.rename(columns={"Within larger district":'District'},inplace=True)
df.head()

,Neighborhood name,District
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,North Beach / Blue Ridge,North Seattle[42]
4,Crown Hill,North Seattle[42]


In [7]:
#  step to change column name to Neighborhood and replace delimeter "/" to ","
df.rename(columns={"Neighborhood name":'Neighborhood'},inplace=True)

df["Neighborhood"] = df["Neighborhood"].str.replace('/',",")
df.head()

,Neighborhood,District
0,North Seattle,Seattle
1,Broadview,North Seattle[42]
2,Bitter Lake,North Seattle[42]
3,"North Beach , Blue Ridge",North Seattle[42]
4,Crown Hill,North Seattle[42]


In [8]:
# Switch columns and clean string numbers like [42](not for now)
df1=df[['District','Neighborhood']]
#df1["District"]=df1["District"].str.replace('\[.*$', '')

print(df1.shape) 
print("Kept the string numerals([42]) to distinguish District from Neighborhood")
df1.head()

(127, 2)
Kept the string numerals([42]) to distinguish District from Neighborhood


,District,Neighborhood
0,Seattle,North Seattle
1,North Seattle[42],Broadview
2,North Seattle[42],Bitter Lake
3,North Seattle[42],"North Beach , Blue Ridge"
4,North Seattle[42],Crown Hill


#### Get Latitude and Longitude Coordinates and Add to Dataframe

In [16]:
# Geocoder
def get_latlng(neighborhood):
    lat_lng_coords = None
    while (lat_lng_coords is None):
        g= geocoder.arcgis('{}, Seattle, Washington'.format(neighborhood))
        lat_lng_coords = g.latlng
        return lat_lng_coords
    

In [17]:
# Get the coordinates and store
coords = [get_latlng(neighborhood) for neighborhood in df1['Neighborhood'].tolist()]


In [18]:
# first ten coordinates
coords[:10]

[[47.643726891883155, -122.3029390149319],
 [47.722380000000044, -122.36497999999995],
 [47.71868000000006, -122.35029999999995],
 [47.70044000000007, -122.38417999999996],
 [47.69520000000006, -122.37409999999994],
 [47.69082000000003, -122.35528999999997],
 [47.713100000000054, -122.31929999999994],
 [47.72320000000008, -122.33869999999996],
 [47.71894000000003, -122.31399999999996],
 [47.69914000000006, -122.33967999999999]]

In [19]:
# Set coords to df2 with column names
df2 = pd.DataFrame(coords, columns=['Latitude','Longitude'])
print(df2.shape)
df2.head()

(127, 2)


,Latitude,Longitude
0,47.643727,-122.302939
1,47.722380,-122.364980
2,47.718680,-122.350300
3,47.700440,-122.384180
4,47.695200,-122.374100


In [20]:
# concat df2 to df1
df3=pd.concat([df1,df2], axis=1)
print(df3.shape)
df3.head()

(127, 4)


,District,Neighborhood,Latitude,Longitude
0,Seattle,North Seattle,47.643727,-122.302939
1,North Seattle[42],Broadview,47.722380,-122.364980
2,North Seattle[42],Bitter Lake,47.718680,-122.350300
3,North Seattle[42],"North Beach , Blue Ridge",47.700440,-122.384180
4,North Seattle[42],Crown Hill,47.695200,-122.374100


In [21]:
# for simplicity working with Districts that contain the word Seattle
seattle = df3[df3['District'].str.contains('Seattle')].reset_index(drop=True)
print(seattle.shape)
seattle.head()

(46, 4)


,District,Neighborhood,Latitude,Longitude
0,Seattle,North Seattle,47.643727,-122.302939
1,North Seattle[42],Broadview,47.722380,-122.364980
2,North Seattle[42],Bitter Lake,47.718680,-122.350300
3,North Seattle[42],"North Beach , Blue Ridge",47.700440,-122.384180
4,North Seattle[42],Crown Hill,47.695200,-122.374100


#### Seattle Map with Neighborhoods

In [22]:
address = 'Seattle, Washington'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


In [25]:
# create map of Toronto using latitude and longitude values

seattle_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(seattle['Latitude'], seattle['Longitude'], seattle['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
         [lat, lng],
      radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(seattle_map)  
    
seattle_map


### Exploring Neighborhoods

##### Explore first Neighborhood

In [27]:
seattle.loc[0,"Neighborhood"]

'North Seattle'

The First Neighborhoods' Latitude and longitude values

In [28]:
neighborhood_latitude = seattle.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = seattle.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = seattle.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of North Seattle are 47.643726891883155, -122.3029390149319.


#### Top 50 venues in 'North Seattle' that are within 500 meters

In [31]:
CLIENT_ID = 'Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3' # your Foursquare ID
CLIENT_SECRET = 'JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3
CLIENT_SECRET:JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52


In [104]:
LIMIT = 50
Radius = 500
Query = "food"
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude, VERSION,Query,Radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=Y0A2AQDZKCRPGODASSROAZKIP0RMZDWC5SWDJWRE4Q3LMSE3&client_secret=JTZBM4N0GCEAOKTEFBM3JJ4N2DB550A1JWZ2TVYLF0NPTU52&ll=47.643726891883155,-122.3029390149319&v=20180605&query=food&radius=500&limit=50'

In [106]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed18b57f7706a001b531965'},
 'response': {'headerLocation': 'Montlake',
  'headerFullLocation': 'Montlake, Seattle',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 47.64822689638316,
    'lng': -122.29627233091648},
   'sw': {'lat': 47.63922688738315, 'lng': -122.30960569894731}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '42951180f964a5203c241fe3',
       'name': 'Cafe Lago',
       'location': {'address': '2305 24th Ave E',
        'lat': 47.6396977895486,
        'lng': -122.30225550533684,
        'labeledLatLngs': [{'label': 'display',
          'lat': 47.6396977895486,
          'lng': -122.30225550533684},
         {'label': 'entrance', 'lat': 47.6

In [107]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [108]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Cafe Lago,Italian Restaurant,47.639698,-122.302256
1,Traveler Montlake,American Restaurant,47.639830,-122.302231
2,Columbia Espresso,Food,47.643642,-122.303859
3,Summer Dog,Hot Dog Joint,47.646608,-122.299445


In [109]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


#### Exploring Neighborhoods in Seattle

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### New Dataframe as seattle_venues

In [41]:
seattle_venues = getNearbyVenues(names=seattle['Neighborhood'],
                                   latitudes=seattle['Latitude'],
                                   longitudes=seattle['Longitude']
                                  )

North Seattle
Broadview
Bitter Lake
North Beach , Blue Ridge
Crown Hill
Greenwood
Northgate
Lake City
Wedgwood
View Ridge
Sand Point
Roosevelt
Ravenna
Bryant
Windermere
Hawthorne Hills
Laurelhurst
University District (U District)
Wallingford
Green Lake
Fremont
Phinney Ridge
Ballard
Central Seattle
Magnolia
Interbay
Queen Anne
Capitol Hill
Montlake
Madison Valley
Madison Park
Lake Union
Downtown
Central Area[120] , Central District ("CD")
South End
West Seattle
Alki Point
North Admiral[153] , Admiral District
Junction[154] , West Seattle Junction , Alaska Junction
Seaview[155] , Mee-Kwa-Mooks
Fairmount Park
Genesee
Gatewood
Fauntleroy
Arbor Heights
Delridge


Dataframe size

In [42]:
print(seattle_venues.shape)
seattle_venues.head()

(902, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Seattle,47.643727,-122.302939,Seattle Public Library - Montlake,47.640520,-122.302413,Library
1,North Seattle,47.643727,-122.302939,Montlake Cut,47.647094,-122.304686,Canal
2,North Seattle,47.643727,-122.302939,Cafe Lago,47.639698,-122.302256,Italian Restaurant
3,North Seattle,47.643727,-122.302939,Fuel Coffee - Montlake,47.639688,-122.302009,Coffee Shop
4,North Seattle,47.643727,-122.302939,Montlake Bicycle Shop,47.639380,-122.302340,Bike Shop


Counting values for each Neighborhood

In [44]:
seattle_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alki Point,5,5,5,5,5,5
Arbor Heights,2,2,2,2,2,2
Ballard,50,50,50,50,50,50
Bitter Lake,27,27,27,27,27,27
Broadview,2,2,2,2,2,2
Bryant,6,6,6,6,6,6
Capitol Hill,46,46,46,46,46,46
"Central Area[120] , Central District (""CD"")",50,50,50,50,50,50
Central Seattle,50,50,50,50,50,50


In [47]:
print('There are {} unique categories'.format(len(seattle_venues['Venue Category'].unique())))


There are 202 unique categories


In [70]:
seattle_venues['Venue Category'].unique()[:50]

array(['Library', 'Canal', 'Italian Restaurant', 'Coffee Shop',
       'Bike Shop', 'Grocery Store', 'American Restaurant', 'Bus Line',
       'Bus Stop', 'Park', 'Salon / Barbershop', 'Harbor / Marina',
       'Trail', 'Concert Hall', 'Donut Shop', 'Food Truck',
       'Thai Restaurant', 'Beer Bar', 'Vietnamese Restaurant',
       'Automotive Shop', 'Sandwich Place', 'Video Store', 'Pizza Place',
       'Convenience Store', 'Sushi Restaurant', 'Fast Food Restaurant',
       'Gas Station', 'Hardware Store', 'Marijuana Dispensary',
       'Intersection', 'Antique Shop', 'Furniture / Home Store', 'ATM',
       'Dive Bar', 'Steakhouse', 'Garden Center', 'Café', 'Burger Joint',
       'Taco Place', 'Sports Bar', 'Greek Restaurant',
       'Thrift / Vintage Store', 'Bank', 'Pet Store', 'Rock Club',
       'Mexican Restaurant', 'Baseball Field', 'Bus Station',
       'Playground', 'Brewery'], dtype=object)

#### Analyze the Data

In [49]:
# one hot encoding
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,Neighborhood,ATM,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Wine Bar,Winery,Yoga Studio
0,North Seattle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,North Seattle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,North Seattle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,North Seattle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,North Seattle,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


New Dataframe size

In [50]:
seattle_onehot.shape

(902, 203)

In [52]:
seattle_grouped=seattle_onehot.groupby("Neighborhood").mean().reset_index()
seattle_grouped

,Neighborhood,ATM,American Restaurant,Amphitheater,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Wine Bar,Winery,Yoga Studio
0,Alki Point,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
1,Arbor Heights,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
2,Ballard,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.040000,0.0000,0.000000,0.0000,0.000000
3,Bitter Lake,0.037037,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.00,0.000,0.00,0.037037,0.037037,0.0000,0.000000,0.0000,0.000000
4,Broadview,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
5,Bryant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
6,Capitol Hill,0.021739,0.065217,0.000000,0.000000,0.000000,0.021739,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.021739
7,"Central Area[120] , Central District (""CD"")",0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000
8,Central Seattle,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.020,0.00,0.000000,0.020000,0.0000,0.000000,0.0000,0.000000
9,Crown Hill,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000


some testing....

In [77]:
"Playground" in seattle_venues['Venue Category'].unique()

True

##### Frequency of venues in neighborhoods

#### Top 5 venues of each neighborhood

In [53]:
num_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alki Point----
                venue  freq
0      Scenic Lookout   0.6
1   Convenience Store   0.2
2  Salon / Barbershop   0.2
3                Park   0.0
4       Movie Theater   0.0


----Arbor Heights----
           venue  freq
0            Spa   0.5
1    Bus Station   0.5
2            ATM   0.0
3   Optical Shop   0.0
4  Movie Theater   0.0


----Ballard----
                venue  freq
0        Cocktail Bar  0.08
1  Mexican Restaurant  0.08
2         Coffee Shop  0.06
3      Ice Cream Shop  0.06
4     Thai Restaurant  0.04


----Bitter Lake----
                  venue  freq
0  Marijuana Dispensary  0.11
1            Food Truck  0.07
2       Automotive Shop  0.07
3       Thai Restaurant  0.07
4                   ATM  0.04


----Broadview----
                 venue  freq
0         Concert Hall   0.5
1                Trail   0.5
2                  ATM   0.0
3      Organic Grocery   0.0
4  Monument / Landmark   0.0


----Bryant----
             venue  freq
0             Café  0.33
1 

##### Change Frequeancy Data to Dataframe

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Top 10 venues of each neighborhood

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alki Point,Scenic Lookout,Convenience Store,Salon / Barbershop,Dive Bar,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space
1,Arbor Heights,Bus Station,Spa,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant
2,Ballard,Cocktail Bar,Mexican Restaurant,Ice Cream Shop,Coffee Shop,Thai Restaurant,Vietnamese Restaurant,Sushi Restaurant,Gym,Dessert Shop,Latin American Restaurant
3,Bitter Lake,Marijuana Dispensary,Thai Restaurant,Food Truck,Automotive Shop,ATM,Convenience Store,Steakhouse,Beer Bar,Sandwich Place,Hardware Store
4,Broadview,Concert Hall,Trail,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space


#### Cluster Neighborhoods
Cluster neighborhoods into four clusters

In [96]:
# set number of clusters
kclusters = 4

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [97]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

seattle_merged = seattle

# merge seattle_grouped with seattle_data to add latitude/longitude for each neighborhood
seattle_merged =seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

seattle_merged.head() 

,District,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Seattle,North Seattle,47.643727,-122.302939,1,Bus Stop,Salon / Barbershop,Coffee Shop,Trail,Library,Canal,Bus Line,Bike Shop,Park,Grocery Store
1,North Seattle[42],Broadview,47.722380,-122.364980,0,Concert Hall,Trail,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space
2,North Seattle[42],Bitter Lake,47.718680,-122.350300,1,Marijuana Dispensary,Thai Restaurant,Food Truck,Automotive Shop,ATM,Convenience Store,Steakhouse,Beer Bar,Sandwich Place,Hardware Store
3,North Seattle[42],"North Beach , Blue Ridge",47.700440,-122.384180,3,Garden Center,Park,Café,Bus Stop,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
4,North Seattle[42],Crown Hill,47.695200,-122.374100,1,Coffee Shop,Pizza Place,Playground,Bank,Rock Club,Bus Station,Sandwich Place,Burger Joint,Mexican Restaurant,Fast Food Restaurant


In [98]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Checking Clusters

Cluster 0

In [100]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 0, seattle_merged.columns[[1] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Broadview,0,Concert Hall,Trail,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space
7,Lake City,0,Trail,Yoga Studio,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space


#### Cluster 1

In [101]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 1, seattle_merged.columns[[1] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North Seattle,1,Bus Stop,Salon / Barbershop,Coffee Shop,Trail,Library,Canal,Bus Line,Bike Shop,Park,Grocery Store
2,Bitter Lake,1,Marijuana Dispensary,Thai Restaurant,Food Truck,Automotive Shop,ATM,Convenience Store,Steakhouse,Beer Bar,Sandwich Place,Hardware Store
4,Crown Hill,1,Coffee Shop,Pizza Place,Playground,Bank,Rock Club,Bus Station,Sandwich Place,Burger Joint,Mexican Restaurant,Fast Food Restaurant
5,Greenwood,1,Coffee Shop,Mexican Restaurant,Bar,Spa,Pizza Place,Bookstore,Supermarket,Steakhouse,Sports Bar,Playground
6,Northgate,1,Arts & Crafts Store,Yoga Studio,Taco Place,Breakfast Spot,Coffee Shop,Department Store,Dog Run,Greek Restaurant,Jewelry Store,Optical Shop
8,Wedgwood,1,Coffee Shop,Soccer Stadium,Video Store,Steakhouse,Dance Studio,Supermarket,Lounge,Yoga Studio,Donut Shop,Fast Food Restaurant
10,Sand Point,1,Convenience Store,Trail,Floating Market,Food Truck,Dog Run,Rugby Pitch,Soccer Field,Theater,Indie Movie Theater,Tennis Court
11,Roosevelt,1,Hobby Shop,Vegetarian / Vegan Restaurant,Latin American Restaurant,Optical Shop,Park,Grocery Store,Gas Station,Coffee Shop,Deli / Bodega,Cupcake Shop
12,Ravenna,1,Mediterranean Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Creperie,Sushi Restaurant,Salon / Barbershop,Bar,Bagel Shop,Café,Bakery
13,Bryant,1,Café,Pizza Place,Grocery Store,Organic Grocery,Donut Shop,Dog Run,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 2

In [102]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 2, seattle_merged.columns[[1] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,South End,2,Playground,Sandwich Place,Dog Run,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space


#### Cluster 3

In [103]:
seattle_merged.loc[seattle_merged['Cluster Labels'] == 3, seattle_merged.columns[[1] + list(range(4, seattle_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"North Beach , Blue Ridge",3,Garden Center,Park,Café,Bus Stop,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
9,View Ridge,3,Bus Stop,Park,Dive Bar,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant
14,Windermere,3,Pizza Place,Beach,Park,Donut Shop,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
15,Hawthorne Hills,3,Playground,Trail,Arts & Crafts Store,Park,Nature Preserve,Dive Bar,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
20,Fremont,3,Park,Bus Station,Bus Stop,Volleyball Court,Winery,Video Store,Spa,Coffee Shop,Convenience Store,Grocery Store
41,Genesee,3,Bus Station,Italian Restaurant,Clothing Store,Park,Yoga Studio,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
42,Gatewood,3,Massage Studio,Coffee Shop,Restaurant,Park,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Exhibit
45,Delridge,3,Park,Coffee Shop,Arts & Crafts Store,Soccer Field,Skate Park,Bus Line,Sandwich Place,Bus Station,Business Service,Gym


How many parks are there in total?

In [110]:
len(seattle_grouped[seattle_grouped["Park"] > 0])

21

##### New Dataframe for parks

In [111]:
seattle_park = seattle_grouped[["Neighborhood","Park"]]
seattle_park.head()

,Neighborhood,Park
0,Alki Point,0.0
1,Arbor Heights,0.0
2,Ballard,0.0
3,Bitter Lake,0.0
4,Broadview,0.0


K-means Clustering

In [117]:
kclusters = 4

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1], dtype=int32)

In [119]:
park_merged = seattle_park.copy()
park_merged["Cluster Labels"] = kmeans.labels_
park_merged.head() 

,Neighborhood,Park,Cluster Labels
0,Alki Point,0.0,1
1,Arbor Heights,0.0,1
2,Ballard,0.0,1
3,Bitter Lake,0.0,1
4,Broadview,0.0,0


In [120]:
park_merged.tail() # we can see park frequncy here!

,Neighborhood,Park,Cluster Labels
41,View Ridge,0.500000,3
42,Wallingford,0.142857,1
43,Wedgwood,0.000000,1
44,West Seattle,0.041667,1
45,Windermere,0.333333,3


In [121]:
park_merged = park_merged.join(seattle.set_index('Neighborhood'), on= 'Neighborhood')

park_merged.head()


,Neighborhood,Park,Cluster Labels,District,Latitude,Longitude
0,Alki Point,0.0,1,West Seattle[151],47.582395,-122.37332
1,Arbor Heights,0.0,1,West Seattle[151],47.508610,-122.37593
2,Ballard,0.0,1,North Seattle[42],47.668670,-122.38453
3,Bitter Lake,0.0,1,North Seattle[42],47.718680,-122.35030
4,Broadview,0.0,0,North Seattle[42],47.722380,-122.36498


In [122]:
park_merged = park_merged.drop(['District'],axis=1)
park_merged.head()

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
0,Alki Point,0.0,1,47.582395,-122.37332
1,Arbor Heights,0.0,1,47.508610,-122.37593
2,Ballard,0.0,1,47.668670,-122.38453
3,Bitter Lake,0.0,1,47.718680,-122.35030
4,Broadview,0.0,0,47.722380,-122.36498


In [123]:
print(park_merged.shape)

(46, 5)


In [127]:
# Sorting by clusters
park_merged1=park_merged.sort_values('Cluster Labels')
park_merged1

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
22,Lake City,0.000000,0,47.717800,-122.283180
4,Broadview,0.000000,0,47.722380,-122.364980
24,Laurelhurst,0.071429,1,47.663980,-122.275820
25,Madison Park,0.045455,1,47.635010,-122.278600
26,Madison Valley,0.000000,1,47.607459,-122.329708
27,Magnolia,0.111111,1,47.633481,-122.387028
28,Montlake,0.000000,1,47.640850,-122.302070
29,"North Admiral[153] , Admiral District",0.000000,1,47.579330,-122.388630
31,North Seattle,0.052632,1,47.643727,-122.302939
32,Northgate,0.000000,1,47.713100,-122.319300


### Second viusalization this time using parks!


In [128]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(park_merged1['Latitude'], park_merged1['Longitude'], park_merged1['Neighborhood'], park_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Now lets examine the cluster based on deciding factor(Parks).

### Cluster 0

In [135]:
park_merged1.loc[park_merged1['Cluster Labels'] == 0]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
22,Lake City,0.0,0,47.71780,-122.28318
4,Broadview,0.0,0,47.72238,-122.36498


### Cluster 1

In [136]:
park_merged1.loc[park_merged1['Cluster Labels'] == 1]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
24,Laurelhurst,0.071429,1,47.663980,-122.275820
25,Madison Park,0.045455,1,47.635010,-122.278600
26,Madison Valley,0.000000,1,47.607459,-122.329708
27,Magnolia,0.111111,1,47.633481,-122.387028
28,Montlake,0.000000,1,47.640850,-122.302070
29,"North Admiral[153] , Admiral District",0.000000,1,47.579330,-122.388630
31,North Seattle,0.052632,1,47.643727,-122.302939
32,Northgate,0.000000,1,47.713100,-122.319300
33,Phinney Ridge,0.000000,1,47.675960,-122.354360
34,Queen Anne,0.000000,1,47.637490,-122.365040


### Cluster 2

In [137]:
park_merged1.loc[park_merged1['Cluster Labels'] == 2]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
39,South End,0.0,2,47.20144,-122.44718


### Cluster 3

In [138]:
park_merged1.loc[park_merged1['Cluster Labels'] == 3]

,Neighborhood,Park,Cluster Labels,Latitude,Longitude
41,View Ridge,0.500000,3,47.680330,-122.272900
10,Delridge,0.307692,3,47.564510,-122.363370
14,Fremont,0.187500,3,47.661370,-122.356080
15,Gatewood,0.250000,3,47.537550,-122.384150
16,Genesee,0.250000,3,47.566630,-122.392890
19,Hawthorne Hills,0.200000,3,47.549275,-122.261729
30,"North Beach , Blue Ridge",0.250000,3,47.700440,-122.384180
45,Windermere,0.333333,3,47.669990,-122.266270


### Observation:
When we grouped neighborhoods into clusters using kmeans clusters we selected the 10 most frequent venues and found that cluster 1 had the most neighborhoods and cluster 2 had the least(only 1) neighborhood.When Building or Buying a house the are a lot of factors that go into buying a home.One of them is how family friendly the neighborhood is, which depends on the venues that are around it.In this case I went with parks! What we saw is that cluster 0 and 2 had no parks and cluster 1 had very few parks even though it had the most neighborhoods.This leaves and with cluster 3 which had the most parks, What is also interesting about neighbourhoods in this cluster is that they are well spread out in the city. We have not taken into consideration other factors such a price which is import but that will require other dataset. But choosing a neighborhood in cluster 3 is the best option here because it is as well spread as cluster 1 and yet had the most parks!

## Thank you!